In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE114783"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE114783"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE114783.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE114783.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE114783.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray gene expression from hepatitis B virus infection to hepatocellular carcinoma"
!Series_summary	"Background: The pathogenesis of hepatitis B virus (HBV)-caused hepatocellular carcinoma (HCC) is complex and not fully understood. In clinical, the effective prevention and treatment of HCC rely on the accurate diagnosis. We developed a biology network approach to investigate the potential mechanisms and biomarkers of each stages from HBV infection to HCC. Methods Global gene profiling of healthy individuals (HC), HBV carriers (HBVC), chronic hepatitis B patients (CHB), liver cirrhosis (LC) and HCC was analyzed by gene array. Differentially expressed genes (DEG) were found by RVM (Random variance model) corrective ANOVA and STC (Series Test of Cluster) analysis."
!Series_overall_design	"peripheral blood mononuclear cells (PBMCs) from 3 healthy individuals (HC),3 HBV carriers (HBVC), 3 chronic hepatitis B patients (CHB), 3 liver cirrhosis (LC) 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray gene expression data
is_gene_available = True

# 2.1 Data Availability
# For trait (Hepatitis status)
trait_row = 0  # The key 0 contains 'diagnosis' which indicates different hepatitis stages

# Age and gender are not explicitly given in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert hepatitis status to binary values (0 for non-hepatitis, 1 for hepatitis-related conditions)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    
    # Healthy control is non-hepatitis
    if 'healthy control' in value:
        return 0
    # All other values indicate hepatitis-related conditions
    elif any(condition in value for condition in ['hepatocellular carcinoma', 'liver cirrhosis', 
                                                'chronic hepatitis b', 'hepatitis b virus carrier']):
        return 1
    else:
        return None

# Define conversion functions for age and gender (not used but needed for function calls)
def convert_age(value):
    """Convert age to float (not used in this dataset)"""
    return None

def convert_gender(value):
    """Convert gender to binary (not used in this dataset)"""
    return None

# 3. Save Metadata
# Initial filtering on usability
is_trait_available = trait_row is not None
is_initial_filtering_passed = validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 30141 rows
First 20 gene IDs:
Index(['AB000409', 'AB000463', 'AB000781', 'AB002294', 'AB002308', 'AB002313',
       'AB002381', 'AB002382', 'AB003177', 'AB003333', 'AB007457', 'AB007870',
       'AB007895', 'AB007921', 'AB007923', 'AB007928', 'AB007937', 'AB007940',
       'AB010419', 'AB010962'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the data are accession numbers (like AB000409), not standard human gene symbols
# These are identifiers for sequences in GenBank/EMBL/DDBJ databases
# They need to be mapped to standard human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # First attempt - use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # The GENE_ID column contains Entrez Gene IDs, not gene symbols
    print("\nThe GENE_ID column contains Entrez Gene IDs, not human gene symbols.")
    
    # Try to find additional annotation information in the SOFT file
    print("\nSearching for more detailed gene annotation in the SOFT file...")
    with gzip.open(soft_file, 'rt') as file:
        # Look for platform ID
        platform_id = None
        for line in file:
            if line.startswith('!Series_platform_id'):
                platform_id = line.split('=')[1].strip()
                print(f"Platform ID: {platform_id}")
                break
    
    # Since we need to map Entrez Gene IDs to gene symbols, we'll need to use an external mapping
    print("\nWe need to map from Entrez Gene IDs to gene symbols.")
    print("Checking if the gene_synonym.json file contains the mapping information...")
    
    # Check if the gene synonym file exists and load it to see its structure
    synonym_path = "./metadata/gene_synonym.json"
    if os.path.exists(synonym_path):
        with open(synonym_path, "r") as f:
            synonym_dict = json.load(f)
        print(f"Found gene synonym dictionary with {len(synonym_dict)} entries.")
        # Show a few sample entries if available
        sample_keys = list(synonym_dict.keys())[:5]
        print(f"Sample entries: {sample_keys}")
    else:
        print("Gene synonym file not found. We'll need another method to map IDs to symbols.")
    
    # Save the gene annotation for later use in mapping
    print("\nSaving gene annotation for mapping step...")
    mapping_data = gene_annotation[['ID', 'GENE_ID']].dropna()
    mapping_data = mapping_data.rename(columns={'GENE_ID': 'Gene'})
    mapping_data['Gene'] = mapping_data['Gene'].astype(int).astype(str)
    
    # Show a preview of the mapping data
    print("Mapping data preview:")
    print(preview_df(mapping_data))
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1130145 rows

Gene annotation preview (first few rows):
{'ID': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'GB_ACC': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'GENE_ID': [8569.0, 6452.0, 85442.0, 6564.0, 9726.0]}

Column names in gene annotation data:
['ID', 'GB_ACC', 'GENE_ID']

The GENE_ID column contains Entrez Gene IDs, not human gene symbols.

Searching for more detailed gene annotation in the SOFT file...
Platform ID: GPL15491

We need to map from Entrez Gene IDs to gene symbols.
Checking if the gene_synonym.json file contains the mapping information...
Gene synonym file not found. We'll need another method to map IDs to symbols.

Saving gene annotation for mapping step...
Mapping data preview:
{'ID': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'Gene': ['8569', '6452', '85442', '6564', '9726']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns to use for mapping
gene_id_column = 'ID'  # The probe ID in gene annotation data 
gene_symbol_column = 'GENE_ID'  # The Entrez Gene ID in gene annotation data

# 2. Get the gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = gene_annotation[['ID', 'GENE_ID']].dropna()
mapping_df = mapping_df.rename(columns={'GENE_ID': 'Gene'})

# Convert Entrez IDs to strings in a format that works with the mapping functions
mapping_df['Gene'] = mapping_df['Gene'].astype(float).astype(int).astype(str)
print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Mapping preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe-level measurements to gene expression data...")
try:
    # Instead of using the extract_human_gene_symbols function which is designed for text,
    # we'll modify our mapping data to work with the existing pipeline
    # Wrap each Entrez ID in a format that mimics gene symbols for the extraction function
    def format_entrez_id(entrez_id):
        # Create a temporary tag that will pass through extract_human_gene_symbols
        return f"ENTREZ{entrez_id}"
    
    # Apply this formatting to create 'Gene' values that will work with the extraction
    mapping_df['Gene'] = mapping_df['Gene'].apply(format_entrez_id)
    
    # Now apply the gene mapping
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Converted gene expression data: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
    
    # 4. Normalize gene symbols if we have gene data
    if gene_data.shape[0] > 0:
        print("\nNormalizing gene symbols to standard format...")
        
        # Apply a function to convert our temporary format back before normalization
        gene_data.index = gene_data.index.str.replace('ENTREZ', '')
        
        # Now normalize using the standard function which will map Entrez IDs to symbols
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {normalized_gene_data.shape[0]} genes × {normalized_gene_data.shape[1]} samples")
        
        # Preview the first few gene symbols after normalization
        print("First 10 normalized gene symbols:")
        print(normalized_gene_data.index[:10].tolist())
        
        # Save the processed gene data
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Processed gene data saved to {out_gene_data_file}")
        
        # Set the final gene data for further processing
        gene_data = normalized_gene_data
    else:
        print("WARNING: No genes were mapped successfully. Gene data is empty.")
        
        # Alternative approach - try using direct mapping instead
        print("\nAttempting direct mapping using original gene annotation...")
        # Use entrez IDs directly as gene IDs for the normalize_gene_symbols_in_index step
        gene_data = gene_annotation.set_index('GENE_ID').join(
            gene_data.T, on='ID', how='inner'
        ).T
        
        if gene_data.shape[0] > 0:
            # Convert index to strings
            gene_data.index = gene_data.index.astype(str)
            
            # Normalize using the standard function
            normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
            print(f"After direct mapping: {normalized_gene_data.shape[0]} genes × {normalized_gene_data.shape[1]} samples")
            
            # Save the processed gene data
            os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
            normalized_gene_data.to_csv(out_gene_data_file)
            print(f"Processed gene data saved to {out_gene_data_file}")
            
            # Set the final gene data for further processing
            gene_data = normalized_gene_data
        else:
            print("WARNING: Alternative mapping also failed. Gene data remains empty.")
            
except Exception as e:
    print(f"Error in gene mapping process: {e}")
    import traceback
    traceback.print_exc()
    print("Unable to complete gene mapping.")

print("\nGene data processing completed.")


Creating gene mapping dataframe...
Created mapping dataframe with 42947 rows
Mapping preview:
{'ID': ['AB000409', 'AB000463', 'AB000781', 'AB001328', 'AB002294'], 'Gene': ['8569', '6452', '85442', '6564', '9726']}

Applying gene mapping to convert probe-level measurements to gene expression data...
Converted gene expression data: 5272 genes × 36 samples

Normalizing gene symbols to standard format...


After normalization: 0 genes × 36 samples
First 10 normalized gene symbols:
[]
Processed gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE114783.csv

Gene data processing completed.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (0, 36)


Normalization resulted in empty dataframe. Using original gene data instead.
Gene data shape after normalization: (0, 36)
Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE114783.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE114783.csv
Linked data shape: (36, 1)

Handling missing values...
After missing value handling, linked data shape: (0, 1)
Skipping bias evaluation due to insufficient data.
Abnormality detected in the cohort: GSE114783. Preprocessing failed.
A new JSON file was created at: ../../output/preprocess/Hepatitis/cohort_info.json

Dataset usability: False
Dataset is not usable for Hepatitis association studies. Data not saved.
